# Lecture 4: Influences & Effects

**Topics**: Influences, Effects, Poincaré Inequality, Edge-Isoperimetric Inequality

**O'Donnell Chapters**: 2.2-2.3

---

## Key Concepts

1. **Influence via Fourier**: $\text{Inf}_i[f] = \sum_{S \ni i} \hat{f}(S)^2$
2. **Total Influence**: $I[f] = \sum_S |S| \cdot \hat{f}(S)^2$
3. **Poincaré Inequality**: $\text{Var}[f] \leq I[f]$
4. **Edge-Isoperimetric**: Relating influences to boundary size

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import boolfunc as bf
from boolfunc.analysis import SpectralAnalyzer

np.random.seed(42)
plt.rcParams['figure.figsize'] = (12, 5)
print("✓ Libraries loaded")

## 1. The Poincaré Inequality

One of the most important inequalities in Boolean function analysis:

$$\text{Var}[f] \leq I[f]$$

where $\text{Var}[f] = \mathbb{E}[f^2] - \mathbb{E}[f]^2 = \sum_{S \neq \emptyset} \hat{f}(S)^2$

**Interpretation**: High variance requires high total influence.

In [ ]:
# Verify Poincaré inequality for various functions
n = 5

functions = {
    "Majority": bf.majority(n),
    "Dictator": bf.dictator(n, 0),
    "Parity": bf.parity(n),
    "AND": bf.AND(n),
    "Tribes(2,4)": bf.tribes(2, 4),
}

print("Verifying Poincaré Inequality: Var[f] ≤ I[f]")
print("=" * 55)
print(f"{'Function':15} | {'Var[f]':>10} | {'I[f]':>10} | Slack")
print("-" * 55)

for name, f in functions.items():
    analyzer = SpectralAnalyzer(f)
    fourier = analyzer.fourier_expansion()
    
    # Var[f] = Σ_{S≠∅} f̂(S)²
    variance = sum(fourier[s]**2 for s in range(1, 2**f.n_vars))
    total_inf = analyzer.total_influence()
    
    slack = total_inf - variance
    check = "✓" if slack >= -1e-10 else "✗"
    print(f"{name:15} | {variance:10.4f} | {total_inf:10.4f} | {slack:.4f} {check}")

## 2. Individual Influences and the KKL Theorem (Preview)

The **KKL Theorem** (Kahn-Kalai-Linial) says: For any balanced function,
$$\max_i \text{Inf}_i[f] \geq \Omega\left(\frac{\log n}{n}\right) \cdot \text{Var}[f]$$

This means: **No low-influence balanced function exists!**

In [ ]:
# Verify KKL for Majority (the most "spread out" influence function)
ns = [3, 5, 7, 9, 11, 13]
max_influences = []
kkl_bounds = []

for n in ns:
    maj = bf.majority(n)
    analyzer = SpectralAnalyzer(maj)
    influences = analyzer.influences()
    
    max_inf = max(influences)
    kkl_bound = np.log(n) / n  # Theoretical lower bound
    
    max_influences.append(max_inf)
    kkl_bounds.append(kkl_bound)

plt.figure(figsize=(10, 5))
plt.plot(ns, max_influences, 'o-', label=r'$\max_i \text{Inf}_i[\text{MAJ}_n]$', linewidth=2)
plt.plot(ns, kkl_bounds, 's--', label=r'KKL bound: $\log(n)/n$', linewidth=2)
plt.xlabel('n (number of voters)')
plt.ylabel('Maximum Influence')
plt.title('KKL Theorem: Max Influence Lower Bound')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Key observation: Majority achieves near-optimal influence distribution!")

## Summary

### Key Takeaways

1. **Poincaré Inequality**: $\text{Var}[f] \leq I[f]$ - high variance needs high influence

2. **KKL Theorem**: $\max_i \text{Inf}_i[f] \geq \Omega(\log n / n)$ for balanced functions

3. **Majority is optimal**: Achieves the lowest max-influence among symmetric balanced functions

### boolfunc Usage
```python
analyzer = SpectralAnalyzer(f)
influences = analyzer.influences()  # All influences
total_inf = analyzer.total_influence()  # I[f]
max_inf = max(influences)  # For KKL check
```